In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data
from sklearn.preprocessing import MinMaxScaler
from itertools import product
import scipy.stats as ss

In [ ]:
device = torch.device('cpu') if not torch.cuda.is_available() else torch.device('cuda')
device

device(type='cuda')

## Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
artists = pd.read_csv("/content/drive/MyDrive/Yoga/studia/semestr6/artists.csv")
tracks = pd.read_json("/content/drive/MyDrive/Yoga/studia/semestr6/tracks.json")
users = pd.read_json("/content/drive/MyDrive/Yoga/studia/semestr6/users.json")
sessions = pd.read_json("/content/drive/MyDrive/Yoga/studia/semestr6/sessions.json")
availible_users = np.load("/content/available_users.npy")

### Tracks

Obróbka danych dotyczących utworów. Za pomocą VALID_COLUMN_NAMES wybierane są do badania atrybuty. Następnie ich numeryczne wartości są normalizowane MinMaxem na przedział [0, 1], podczas gdy kategoryczne i w tym konkretnym wypadku klucz pozostają bez zmian

In [ ]:
# select = sessions['track_id'].value_counts().to_frame()
# weights = [select.loc[i].to_numpy()[0] if i in select.index else 0 for i in tracks['id']]
# len(weights)

In [ ]:
# tracks = tracks.sample(n = 250, weights=weights).reset_index(drop=True)

In [ ]:
track_ids = tracks.id
VALID_COLUMN_NAMES = ['id', 'duration_ms', 'popularity', 'explicit', 'release_date','danceability', 'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
tracks = tracks[VALID_COLUMN_NAMES]
rd = tracks.release_date
rd = pd.to_datetime(rd, errors='coerce')
tracks['release_date'] = rd.dt.year.fillna(0).astype(int)

<ipython-input-7-9fd096d18458>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tracks['release_date'] = rd.dt.year.fillna(0).astype(int)


In [ ]:
tracks = tracks.reset_index(drop=True)
tracks.head()

,id,duration_ms,popularity,explicit,release_date,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0RNxWy0PC3AyH4ThH3aGK6,201467,55,0,1929,0.673,0.3770,0,-14.141,0.0697,0.586,0.000000,0.332,0.713,88.973
1,2W889aLIKxULEefrleFBFI,198000,54,0,1943,0.204,0.1510,2,-17.842,0.0418,0.947,0.000009,0.321,0.134,91.783
2,4Pnzw1nLOpDNV6MKI5ueIR,199000,55,0,1944,0.295,0.0826,1,-19.569,0.0367,0.984,0.000358,0.156,0.169,128.600
3,7GLmfKOe5BfOXk7334DoKt,163000,54,0,1944,0.561,0.3350,9,-11.093,0.0499,0.840,0.000002,0.788,0.590,126.974
4,6kD1SNGPkfX9LwaGd1FG92,186173,53,0,1944,0.197,0.0546,1,-22.411,0.0346,0.950,0.276000,0.152,0.100,90.150


In [ ]:
track_keys = tracks['key']
tracks_scaler = MinMaxScaler()
tracks_scaler.fit(tracks.drop(columns=['id', 'key']))
tracks = tracks_scaler.transform(tracks.drop(columns=['id', 'key']))
tracks = pd.DataFrame(tracks, columns=['duration_ms', 'popularity', 'explicit', 'release_date','danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo'])
tracks['id'] = track_ids
tracks['key'] = track_keys
tracks.head()

,duration_ms,popularity,explicit,release_date,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,key
0,0.041775,0.083333,0.0,0.000000,0.686735,0.377313,0.671868,0.073835,0.588353,0.000000,0.324873,0.719475,0.404241,0RNxWy0PC3AyH4ThH3aGK6,0
1,0.040927,0.062500,0.0,0.152174,0.208163,0.151064,0.589719,0.044280,0.950803,0.000009,0.313706,0.135217,0.417008,2W889aLIKxULEefrleFBFI,2
2,0.041172,0.083333,0.0,0.163043,0.301020,0.082588,0.551385,0.038877,0.987952,0.000359,0.146193,0.170535,0.584283,4Pnzw1nLOpDNV6MKI5ueIR,1
3,0.032369,0.062500,0.0,0.163043,0.572449,0.335267,0.739523,0.052860,0.843373,0.000002,0.787817,0.595358,0.576895,7GLmfKOe5BfOXk7334DoKt,9
4,0.038035,0.041667,0.0,0.163043,0.201020,0.054557,0.488302,0.036653,0.953815,0.276553,0.142132,0.100908,0.409588,6kD1SNGPkfX9LwaGd1FG92,1


In [ ]:
import joblib
joblib.dump(tracks_scaler, "classifier_track.scaler")

['classifier_track.scaler']

### Users

Obróbka danych dotyczących użytkowników. Usuwane są kolumny zadeklarowane jako NON_VALID oraz rekordy użytkowników nie należących do whitelisty znajdującej się w pliku "availible users". Na podstawie pełnych danych zostaje wyprodukowana lista gatunków.

In [ ]:
GENRES = np.unique(np.concatenate(users['favourite_genres'].to_numpy()))
NON_VALID_USER_ATRIBUTES = ['name', 'city', 'street']

In [ ]:
#users = users.sample(n = 500).reset_index(drop=True)
users = users[users['user_id'].isin(availible_users)].reset_index(drop=True)

In [ ]:
users = users.drop(columns=NON_VALID_USER_ATRIBUTES)
#GENRES = np.unique(np.concatenate(users['favourite_genres'].to_numpy()))

In [ ]:
# users = users[:10]
users = users.reset_index(drop=True)
users.head()

,user_id,favourite_genres,premium_user
0,141,"[k-pop boy group, alternative rock, ranchera]",True
1,167,"[regional mexican, pop punk, album rock]",True
2,200,"[corrido, post-teen pop, canadian pop]",True
3,206,"[mexican pop, folk rock, k-pop boy group]",True
4,237,"[permanent wave, k-pop boy group, post-teen pop]",True


### Sessions
Obróbka danych dotyczących sesji. Zostają usunięte rekordy dotyczące użytkowników spoza whitelisty avalilible_users. Model nie chce przewidywać również rekordów typu 'skip' oraz 'advertisment'. W wypadku czyszczenia z tych pierwszych, czyszczony jest także rekord 'play' go poprzedzający (występują w parach). Do celów treningowych zostają wykorzystane tylko i wyłącznie dane poprzedzające o co najmniej X_MONTHS_AB miesięcy - owe miesiące, które model nie widzi ani w trakcie nauki ani walidacji są traktowane jako testowe i zostaną następnie wykorzystane jako dane do testów AB.

In [ ]:
sessions = sessions[sessions['user_id'].isin(availible_users)].reset_index(drop=True)

In [ ]:
to_drop = sessions['event_type'] == 'skip'
to_drop[len(to_drop)] = False
to_drop = [ (to_drop[i+1] or to_drop[i]) for i in range(len(to_drop) - 1) ]
to_drop = np.array(to_drop)
to_drop += sessions['event_type'] == 'advertisment'
to_drop = ~to_drop
sessions = sessions[to_drop]

In [ ]:
pivot = sessions['timestamp'].max() - pd.tseries.offsets.DateOffset(months=1)
sessions = sessions[sessions['timestamp'] < pivot].reset_index(drop=True)

In [ ]:
sessions = sessions.reset_index(drop=True)
sessions.head()

,session_id,timestamp,user_id,track_id,event_type
0,24172,2021-11-06 08:29:36.000,141,4mVOwzuPKrMb4CDVe9Q4Hs,play
1,24172,2021-11-06 08:33:31.733,141,6GQLX6Z28fYwDNCrhaKzYF,play
2,24172,2021-11-06 08:36:23.306,141,63xBnyUVKIupzjHno4wFs3,play
3,24172,2021-11-06 08:40:11.639,141,4P4s2KHOw0uISbLI3zkHtD,play
4,24172,2021-11-06 08:44:49.478,141,4P4s2KHOw0uISbLI3zkHtD,like


## Sprawdzenie istotności i selekcja atrybutów

Utworzono strukturę Dataset która na podstawie zadanych tabel tworzy przypadki testowe dla modelu. Następnie na podstawie reprezentatywnej próby nieobciążonej wygenerowanych przypadków zbadano istotność poszczególnych atrybutów.

In [ ]:
class MusicDataset(data.Dataset):
    def __init__(self, users, tracks, sessions):
        self.users = users
        self.lusers = len(users)
        self.tracks = tracks
        self.ltracks = len(tracks)
        self.sessions = sessions
        self.z = max(self.lusers, self.ltracks)

    def __len__(self):
        return self.ltracks * self.lusers

    def __getitem__(self, idx):
        u = users.loc[idx // self.z]
        t = tracks.loc[idx % self.z]
        select = sessions[u['user_id'] == sessions['user_id']]
        select = select[t['id'] == select['track_id']]
        play = bool(sum(select['event_type'] == 'play'))
        like = bool(sum(select['event_type'] == 'like'))
        ug = u['favourite_genres']
        ug = torch.Tensor(np.array([i in ug for i in GENRES]) * 1)
        ur = torch.Tensor([u['premium_user'] * 1])
        tk = torch.Tensor([int(i == t['key']) for i in range(16)])
        tr = torch.Tensor(t.drop(labels=['id', 'key']).to_numpy().astype(np.float64))

        return [[ug, ur], [tk, tr]], [play, like]

In [ ]:
def verify_cross_significance_categorical(X): # categorical values data frame
  matrix = list(product(list(X.columns), list(X.columns), repeat=1))
  result = []
  for i in matrix:
    if i[0] != i[1]:
      result.append((i[0],i[1],list(ss.chi2_contingency(pd.crosstab(X[i[0]], X[i[1]])))[1]))
  chi_test_output = pd.DataFrame(result, columns = ['var1', 'var2', 'coeff'])
  return chi_test_output.pivot(index='var1', columns='var2', values='coeff')

In [ ]:
STATISTIC_SAMPLE = 100000
dataset = MusicDataset(users, tracks, sessions)
dataloader = data.DataLoader(dataset, batch_size=1, shuffle=True, drop_last=True)
# [0][0][0][0] - favourite genres embed; [0][0][1][0] - premium embed; [0][1][0][0] - key embed; [0][1][1][0] - rest param; s[1][0] - play; s[1][1] - like
STATISTIC_SAMPLE = min(STATISTIC_SAMPLE, len(dataset))

genres = []
premium = []
key = []
rest = []

i = 0

for s in dataloader: # taking sample of 100k examples
    genres.append(np.concatenate((s[0][0][0][0].numpy(), s[1][0].numpy(), s[1][1].numpy())))
    premium.append(np.concatenate((s[0][0][1][0].numpy(), s[1][0].numpy(), s[1][1].numpy())))
    key.append(np.concatenate((s[0][1][0][0].numpy(), s[1][0].numpy(), s[1][1].numpy())))
    rest.append(np.concatenate((s[0][1][1][0].numpy(), s[1][0].numpy(), s[1][1].numpy())))
    if i > STATISTIC_SAMPLE:
      break;
    else:
      i += 1

In [ ]:
genres = pd.DataFrame(genres, columns=np.concatenate((GENRES, ['PLAYED', 'LIKED'])))
ch_genres = verify_cross_significance_categorical(genres)
ch_genres

var2,LIKED,PLAYED,adult standards,album rock,alternative metal,alternative rock,art rock,canadian pop,classic rock,contemporary country,...,rap,reggaeton,regional mexican,rock,rock en espanol,soft rock,trap latino,tropical,uk pop,urban contemporary
var1,,,,,,,,,,,,,,,,,,,,,
LIKED,NaN,0.000000e+00,4.267394e-01,1.754712e-05,1.0,9.392939e-02,3.770380e-05,7.377324e-30,1.0,1.0,...,2.642716e-10,6.546859e-07,2.619476e-02,1.0,2.642716e-10,9.632678e-22,1.010264e-07,1.0,3.660868e-05,3.756353e-12
PLAYED,0.000000e+00,NaN,1.084962e-05,3.805908e-22,1.0,1.222215e-03,7.974330e-01,7.983340e-87,1.0,1.0,...,8.931798e-09,1.101024e-08,2.082377e-06,1.0,8.931798e-09,1.366040e-106,1.159901e-41,1.0,8.874424e-02,7.210214e-04
adult standards,4.267394e-01,1.084962e-05,NaN,3.766591e-64,1.0,1.164686e-129,5.040931e-63,8.940646e-130,1.0,1.0,...,4.609958e-63,5.930505e-131,1.683987e-131,1.0,4.609958e-63,1.799396e-131,1.038607e-63,1.0,5.618118e-62,1.776379e-63
album rock,1.754712e-05,3.805908e-22,3.766591e-64,NaN,1.0,1.398562e-129,5.503913e-63,1.073774e-129,1.0,1.0,...,5.033636e-63,7.134552e-131,0.000000e+00,1.0,5.033636e-63,2.166322e-131,1.135108e-63,1.0,6.124960e-62,1.940783e-63
alternative metal,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,NaN,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.0,...,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.000000e+00,1.000000e+00
alternative rock,9.392939e-02,1.222215e-03,1.164686e-129,1.398562e-129,1.0,NaN,2.807023e-127,8.706405e-264,1.0,1.0,...,2.338386e-127,3.383420e-266,2.575250e-267,1.0,2.338386e-127,2.949261e-267,1.111804e-128,1.0,3.875165e-125,3.329857e-128
art rock,3.770380e-05,7.974330e-01,5.040931e-63,5.503913e-63,1.0,2.807023e-127,NaN,2.165416e-127,1.0,1.0,...,6.430279e-62,1.510685e-128,4.391248e-129,1.0,6.430279e-62,0.000000e+00,1.489354e-62,1.0,7.481164e-61,2.522062e-62
canadian pop,7.377324e-30,7.983340e-87,8.940646e-130,1.073774e-129,1.0,8.706405e-264,2.165416e-127,NaN,1.0,1.0,...,1.803601e-127,1.959057e-266,1.487671e-267,1.0,1.803601e-127,1.703938e-267,8.551977e-129,1.0,3.002645e-125,2.563841e-128
classic rock,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.000000e+00,1.000000e+00,1.000000e+00,NaN,1.0,...,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.000000e+00,1.000000e+00,1.000000e+00,1.0,1.000000e+00,1.000000e+00


In [ ]:
premium = pd.DataFrame(premium, columns=['premium', 'PLAYED', 'LIKED'])
ch_premium = verify_cross_significance_categorical(premium)
ch_premium

var2,LIKED,PLAYED,premium
var1,,,
LIKED,NaN,0.0,1.0
PLAYED,0.0,NaN,1.0
premium,1.0,1.0,NaN


results indicate that *premium* cocefficiant is insignificant

In [ ]:
key = pd.DataFrame(key, columns=list(range(len(key[1]) - 2)) + ['PLAYED', 'LIKED'])
ch_key = verify_cross_significance_categorical(key)
ch_key

var2,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,LIKED,PLAYED
var1,,,,,,,,,,,,,,,,,,
0,NaN,4.555956e-289,0.000000e+00,2.496666e-99,1.230214e-257,4.988427e-255,4.376894e-211,0.000000e+00,1.118521e-196,0.000000e+00,4.071807e-201,3.761315e-249,1.0,1.0,1.0,1.0,0.165298,0.058337
1,4.555956e-289,NaN,5.871436e-253,2.036709e-76,1.256844e-197,1.246880e-195,5.463710e-162,6.064410e-270,5.859893e-151,1.785000e-262,2.341325e-154,3.937990e-191,1.0,1.0,1.0,1.0,0.299637,0.055547
2,0.000000e+00,5.871436e-253,NaN,4.732002e-87,1.780073e-225,3.394029e-223,8.968901e-185,4.519538e-308,3.550186e-172,1.533463e-299,4.666797e-176,4.672861e-218,1.0,1.0,1.0,1.0,0.025860,0.005511
3,2.496666e-99,2.036709e-76,4.732002e-87,NaN,3.118118e-68,1.489809e-67,4.176427e-56,7.932170e-93,2.374161e-52,2.747352e-90,1.654327e-53,5.057440e-66,1.0,1.0,1.0,1.0,0.012055,0.000389
4,1.230214e-257,1.256844e-197,1.780073e-225,3.118118e-68,NaN,2.033300e-174,1.869422e-144,1.325385e-240,1.248322e-134,5.946544e-234,1.173230e-137,2.068539e-170,1.0,1.0,1.0,1.0,0.239148,0.873064
5,4.988427e-255,1.246880e-195,3.394029e-223,1.489809e-67,2.033300e-174,NaN,5.329034e-143,3.603840e-238,2.826342e-133,1.383273e-231,2.851695e-136,1.083826e-168,1.0,1.0,1.0,1.0,0.634142,0.444817
6,4.376894e-211,5.463710e-162,8.968901e-185,4.176427e-56,1.869422e-144,5.329034e-143,NaN,3.765621e-197,1.916191e-110,1.042368e-191,6.391527e-113,1.012875e-139,1.0,1.0,1.0,1.0,0.004166,0.000170
7,0.000000e+00,6.064410e-270,4.519538e-308,7.932170e-93,1.325385e-240,3.603840e-238,3.765621e-197,NaN,1.058569e-183,0.000000e+00,7.606619e-188,1.104109e-232,1.0,1.0,1.0,1.0,0.067932,0.019766
8,1.118521e-196,5.859893e-151,3.550186e-172,2.374161e-52,1.248322e-134,2.826342e-133,1.916191e-110,1.058569e-183,NaN,1.237496e-178,2.899556e-105,3.196518e-130,1.0,1.0,1.0,1.0,0.011144,0.000034


Tests indicate *key* to be a significant variable

In [ ]:
rest = pd.DataFrame(rest, columns=list(list(dataset.tracks.drop(columns=['id', 'key']).columns) + ['PLAYED', 'LIKED']))
rest.corr(method='spearman')

,duration_ms,popularity,explicit,release_date,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,PLAYED,LIKED
duration_ms,1.000000,-0.106160,-0.096622,-0.283085,-0.210223,0.016163,-0.080405,-0.187939,-0.107949,0.186694,-0.007682,-0.211614,-0.026060,0.046918,0.030580
popularity,-0.106160,1.000000,0.117389,0.283519,0.092429,-0.038556,0.067142,0.061841,-0.019966,-0.087255,-0.037187,-0.022745,0.003949,0.014797,0.009361
explicit,-0.096622,0.117389,1.000000,0.247943,0.212666,0.023089,0.038151,0.390392,-0.055674,-0.080620,0.014160,-0.069773,-0.015155,-0.022003,-0.008471
release_date,-0.283085,0.283519,0.247943,1.000000,0.242373,-0.016803,0.257948,0.254779,0.086401,-0.285189,-0.004773,-0.097451,0.028349,-0.144020,-0.089989
danceability,-0.210223,0.092429,0.212666,0.242373,1.000000,0.005818,0.074906,0.229128,0.043484,-0.127868,-0.106232,0.452230,-0.118684,-0.036303,-0.024068
energy,0.016163,-0.038556,0.023089,-0.016803,0.005818,1.000000,0.707340,0.289041,-0.606946,-0.020092,0.139890,0.315833,0.166778,0.000474,0.002313
loudness,-0.080405,0.067142,0.038151,0.257948,0.074906,0.707340,1.000000,0.192857,-0.424782,-0.266528,0.097220,0.209758,0.134971,-0.012955,-0.004474
speechiness,-0.187939,0.061841,0.390392,0.254779,0.229128,0.289041,0.192857,1.000000,-0.142247,-0.131547,0.047257,0.123562,0.093958,-0.047488,-0.025064
acousticness,-0.107949,-0.019966,-0.055674,0.086401,0.043484,-0.606946,-0.424782,-0.142247,1.000000,-0.110788,-0.046800,-0.079648,-0.120218,-0.049223,-0.032996
instrumentalness,0.186694,-0.087255,-0.080620,-0.285189,-0.127868,-0.020092,-0.266528,-0.131547,-0.110788,1.000000,-0.067674,-0.112459,-0.016757,0.021332,0.015346


## Model

Strojenie hiperparametrów metodą Random Search (efektywność wyższa niż Grid Search [1]). Strojone HP: 
* neurons per layer,
* dropout rate

Strojenie wg. naiwnego accuracy na zbiorze walidacyjnym.

| Test no | Layer 1 | Dropout 1 | Layer 2 | Dropout 2 | Layer 3 | Dropout 3 | Layer 4 | Accuracy play | Accuracy like |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| **1** | 128 | 0.1 | 64 | - | - | - | - | 7.8% | 54.7% |
| **2** | 128 | 0.1 | 64 | 0.1 | 32 | - | - | 13.1% | 78.2% |
| **3** | 128 | 0 | 64 | 0.1 | 32 | 0 | 32 | 20.3% | 82.1% |
| **4** | 256 | 0.1 | 128 | 0.1 | 64 | 0.1 | 32 | 18.1% | 92.2% |

Iteracji jest relatywnie mało względu na długi trening modeli (rzędu 5h na lokalnym sprzęcie).




*\[1]Liashchynskyi, Petro, and Pavlo Liashchynskyi. "Grid search, random search, genetic algorithm: a big comparison for NAS." arXiv preprint arXiv:1912.06059 (2019).*

Classifier input: <br>
    1. Track Desccription (12 numerical + 2 categorical giving 16 one-hots + 1 binary) <br>
    2. User Description (2 categorical giving 50 one-hots + 1 binary) <br>
Classifier output: <br>
    1. P(User will listen whole track) <br>
    2. P(User will like the track) <br>

In [ ]:
class Music_classifier(nn.Module):
    def __init__(self, genres):
        self.genres = genres
        #    |
        # 50 | Genres | 10    }
        #    |                } USER [11]
        # czy_premium | 1     }
    
        #    |
        # 16 | key    | 3     }
        #    |                } TRACK [16]
        # numeric     | 12    }
        # binary      | 1     }
    
        # TRACK [16] |        }
        # TRACK [16] |   | 10 } FINGERPRINT [10]
        # TRACK [16] |        }
        track_key_code = 3
        user_genre_code = 10
        fingerprint_params = 15
        super(Music_classifier, self).__init__()
        ##### USER PREP ######
        self.emb_user_genre = nn.Linear(len(genres), user_genre_code)
        self.emb_user_genre_act = nn.LeakyReLU()

        ##### TRACK PREP #####
        self.emb_track_key = nn.Linear(16, track_key_code)
        self.emb_track_key_act = nn.LeakyReLU()

        ##### FINGERPRINT PREP #####
        # self.emb_fingerprint = nn.Linear(3 * (track_key_code + 12 + 1), fingerprint_params)
        # self.emb_fingerprint_act = nn.LeakyReLU()
        # self.emb_fingerprint_track1 = nn.Linear(16, track_key_code)
        # self.emb_fingerprint_track1_act = nn.LeakyReLU()
        # self.emb_fingerprint_track2 = nn.Linear(16, track_key_code)
        # self.emb_fingerprint_track2_act = nn.LeakyReLU()
        # self.emb_fingerprint_track3 = nn.Linear(16, track_key_code)
        # self.emb_fingerprint_track3_act = nn.LeakyReLU()

        #### MAIN CLASSIFIER ####
        user_params = user_genre_code + 1
        track_params = track_key_code + 12 + 1

        self.layers = nn.Sequential(
            # nn.Linear(user_params + track_params + fingerprint_params, 256),
            nn.Linear(user_params + track_params, 128),
            nn.LeakyReLU(),

            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.Dropout(0.1),
            nn.LeakyReLU(),

            nn.Linear(64, 32),
            nn.ReLU(),
            #nn.Linear(32, 3),
            nn.Linear(32, 2),

            nn.Sigmoid()
        )
    #def forward(self, user, track, finger):
    def forward(self, x):
        # user -> [Tensor(50, bs), Tensor(1, bs)]
        # track -> [Tensor(16, bs), Tensor(13, bs)]
        # finger -> [ [Tensor(16, bs), Tensor(13, bs)], [Tensor(16, bs), Tensor(13, bs)], [Tensor(16, bs), Tensor(13, bs)] ]
        user, track = x

        comp_emb_user_genre = self.emb_user_genre(user[0])
        comp_emb_user_genre = self.emb_user_genre_act(comp_emb_user_genre)

        comp_emb_track_key = self.emb_track_key(track[0])
        comp_emb_track_key = self.emb_track_key_act(comp_emb_track_key)

        # comp_emb_fingerprint_track1 = self.emb_fingerprint_track1(finger[0][0])
        # comp_emb_fingerprint_track1 = self.emb_fingerprint_track1_act(comp_emb_fingerprint_track1)
        # comp_emb_fingerprint_track2 = self.emb_fingerprint_track2(finger[1][0])
        # comp_emb_fingerprint_track2 = self.emb_fingerprint_track2_act(comp_emb_fingerprint_track2)
        # comp_emb_fingerprint_track3 = self.emb_fingerprint_track3(finger[2][0])
        # comp_emb_fingerprint_track3 = self.emb_fingerprint_track3_act(comp_emb_fingerprint_track3)
        # emb_finger_x = torch.cat([comp_emb_fingerprint_track1, finger[0][1], 
        #                           comp_emb_fingerprint_track2, finger[1][1], 
        #                           comp_emb_fingerprint_track3, finger[2][1]], dim=1)
        # comp_emb_fingerprint = self.emb_fingerprint(emb_finger_x)
        # comp_emb_fingerprint = self.emb_fingerprint_act(comp_emb_fingerprint)

        # x = torch.cat([comp_emb_user_genre, user[1], comp_emb_track_key, track[1], comp_emb_fingerprint])
        # print(comp_emb_user_genre.shape, user[1].shape, comp_emb_track_key.shape, track[1].shape)
        c = torch.cat([comp_emb_user_genre, user[1], comp_emb_track_key, track[1]], 1)
        
        return self.layers(c)


## Training

Naiwna funkcja accuracy *accuracy(loader, model, device)* w celach testowych.
* Optymalizator ADAM z krokiem 0.001 (mały krok ze względu na normalizację danych)
* Funkcja straty w postaci Krzyżowej Entropii (klasyfikacja binarna)

In [ ]:
def accuracy(loader, model, device):
    with torch.no_grad():
        good_like = 0
        good_play = 0
        all = 0
        for x, y in loader:
            x = [ [ x[0][0].to(device), x[0][1].to(device) ], [ x[1][0].to(device), x[1][1].to(device) ] ]
            y = [ y[0].to(device), y[1].to(device) ]  
            preds = model(x)
            preds = preds.to(device)
            good_play = sum([a == b for a, b in zip(np.round(preds.cpu().detach().numpy(), 0).T[0], y[0])])
            good_like = sum([a == b for a, b in zip(np.round(preds.cpu().detach().numpy(), 0).T[1], y[1])])
            all = len(preds)
        print(f"Accuracy of playing: {good_play * 100/all:.3}%\nAccuracy of liking: {good_like * 100/all:.3}%\n")
           

In [ ]:
dataset = MusicDataset(users, tracks, sessions)
train, valid = data.random_split(dataset, [round(len(dataset) * 0.7), round(len(dataset) * 0.3)])

In [ ]:
train_dataloader = data.DataLoader(train, batch_size=64, shuffle=True, drop_last=True)
valid_dataloader = data.DataLoader(valid, batch_size=64, shuffle=False, drop_last=False)

In [ ]:
model = Music_classifier(GENRES)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_module = nn.CrossEntropyLoss()

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/Yoga/studia/semestr6/classifier.model'))

<All keys matched successfully>

In [ ]:
model.to(device)

Music_classifier(
  (emb_user_genre): Linear(in_features=50, out_features=10, bias=True)
  (emb_user_genre_act): LeakyReLU(negative_slope=0.01)
  (emb_track_key): Linear(in_features=16, out_features=3, bias=True)
  (emb_track_key_act): LeakyReLU(negative_slope=0.01)
  (layers): Sequential(
    (0): Linear(in_features=27, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.01)
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): Dropout(p=0.1, inplace=False)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=2, bias=True)
    (9): Sigmoid()
  )
)

In [ ]:
EPOCHS = 10
model.train()
for epoch in range(EPOCHS):
    for x, y in train_dataloader:
        x = [ [ x[0][0].to(device), x[0][1].to(device) ], [ x[1][0].to(device), x[1][1].to(device) ] ]
        y = [ y[0].to(device), y[1].to(device) ]  
        preds = model(x)
        loss = loss_module(preds, torch.cat([y[0].unsqueeze(1), y[1].unsqueeze(1)], 1).float())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch}, loss: {loss.sum().item():.3}')
    # torch.save(model.state_dict(), '/content/drive/MyDrive/Yoga/studia/semestr6/classifier.model')
    if(epoch % 5 == 0):
        accuracy(train_dataloader, model, device)


Epoch: 0, loss: 0.117
Accuracy of playing: 14.1%
Accuracy of liking: 81.2%

Epoch: 1, loss: 0.12
Epoch: 2, loss: 0.0815
Epoch: 3, loss: 0.166
Epoch: 4, loss: 0.0552


In [ ]:
accuracy(valid_dataloader, model, device)

Accuracy of playing: 25.0%
Accuracy of liking: 75.0%



In [ ]:
accuracy(valid_dataloader, lambda x: torch.zeros(64, 2), device)

Accuracy of playing: 9.38%
Accuracy of liking: 10.9%



## Saving

Zapisywanie rezultatów sesji.

In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/Yoga/studia/semestr6/classifier.model')

In [ ]:
import joblib
joblib.dump(tracks_scaler, "classifier_track.scaler")

In [ ]:
len(tracks) * len(users) / 60 / 64